In [1]:
import re
import string
import requests
import tempfile
from bs4 import BeautifulSoup

In [2]:
import clean_text_class as clean
import count_words_letters_class as count

In [3]:
base_url = "http://www.gutenberg.org/wiki/Category:"
# categories = ["Bookshelf", "FR_Genre", "DE_Bücherregal", "IT_Biblioteca"]
categories = {"English": "Bookshelf", "français": "FR_Genre", "Deutsch": "DE_Bücherregal", "italiano": "IT_Biblioteca"}
store_subcategories = [base_url + category for category in list(categories.values())]
# Write results as a dict into a list
languages_ebooks = []

In [4]:
def analyse_text(book, result):
    raw = clean.clean_text(book)
    text = raw.remove_metadata(raw.text)
    content = raw.remove_chapters(text)
    content = raw.remove_punctuation(content)
    letters = raw.text_letters(content)
    analysis = count.analyse_letters(letters)
    letters_count = analysis.letters_count(analysis.letters)
    letters_freq = analysis.letters_freq_pd(letters_count)
    result_freq = {**result, **letters_freq}
    return result_freq

### Gather subcategories of books for each language then get links to ebooks texts for each subcategory

In [ ]:
# test_url = "http://www.gutenberg.org/wiki/Category:DE_B%C3%BCcherregal", test_url2 = "http://www.gutenberg.org/wiki/DE_Drama_(B%C3%BCcherregal)"

for language, category in categories.items():
    url = base_url + category
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    sub_links = [f"http://www.gutenberg.org{a['href']}" for a in soup.find("div", {"class":"mw-content-ltr"})("a")]
    # store_subcategories.extend(sub_links)
    for link in sub_links:
        # print(link)
        store_subcategories.append(link)
        if "How-To" in link:
            pass
        elif "Category" in link:
            more_links = [f"http://www.gutenberg.org{a['href']}" for a in soup.find("div", {"class":"mw-content-ltr"})("a")]
            # store_subcategories.extend(more_links)
            new_links = [l for l in more_links if l not in store_subcategories]
            sub_links.extend(more_links)
        else:
            # print("> ELSE")
            r2 = requests.get(link)
            r2.status_code
            soup2 = BeautifulSoup(r2.text, 'html.parser')
            ebooks = [f"http:{a['href']}.txt.utf-8" for a in soup2("a", {"class": "extiw"}) if "ebooks" in a['href']]
            if not ebooks:
                print("No ebook")
            else:
                for ebook in ebooks:
                    result = {}
                    result["language"] = language
                    id = [e for e in re.split('[/.]', ebook) if e.isdigit()]
                    # print(id)
                    try:
                        result["id"] = id[0]
                    except IndexError:
                        result["id"] = "N/A"
                        print("No ebook")
                        #print(f"link: {link}")
                        #print(f" > ebooks: {ebooks}")
                    r3 = requests.get(ebook)
                    tmpf = tempfile.NamedTemporaryFile()
                    with open(tmpf.name, 'w') as f:
                        f.write(r3.text)
                    res = analyse_text(tmpf.name, result)
                    languages_ebooks.append(res)

In [ ]:
languages_ebooks[0]

### Content from Project Gutenberg

English: 15846 ebooks

français: 3228 ebooks

Deutsch: 916 ebooks

italiano: 785 ebooks

In [ ]:
import pandas as pd

In [ ]:
letters_pd = pd.DataFrame(languages_ebooks)